In [1]:
from google.colab import drive
drive.mount('/content/drive') #Syncing colab with drive

Mounted at /content/drive


In [13]:
#Importing the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR
import math
from fbprophet import Prophet
import warnings
warnings.filterwarnings('ignore')

In [3]:
%cd '/content/drive/My Drive/EXL 2021/EXL'

/content/drive/My Drive/EXL 2021/EXL


In [4]:
#Loading the Dataset
train_data = pd.read_csv('data_preprocessed.csv')
test_data = pd.read_csv('validation_data.csv')

In [5]:
train_data['C_TOT_POP'] = (train_data['confirmed_cases']/train_data['C_TOT_POP'])*100

In [6]:
#Grouping Data by County
data_by_county = train_data.groupby('countyFIPS')

In [73]:
df = data_by_county.get_group(1001)
df['date'] = pd.to_datetime(df['date'])
df = df.set_index(df['date'])

In [74]:
#Selecting some features
df = df[['confirmed_cases', 'deaths','target', 'google_mobility_retail_and_recreation',
       'google_mobility_grocery_and_pharmacy', 'google_mobility_workplaces','C_TOT_POP',
       'google_mobility_residential', 'apple_mobility_driving','test_count',
       'test_rate', 'new_test_rate', 'new_test_count','critical_staffing_shortage_today_yes_SD',
       'critical_staffing_shortage_anticipated_within_week_yes_SD',
       'hospital_onset_covid_SD', 'hospital_onset_covid_coverage_SD',
       'inpatient_beds_SD', 'inpatient_beds_coverage_SD',
       'previous_day_admission_adult_covid_confirmed_SD',
       'previous_day_admission_adult_covid_suspected_SD',
       'previous_day_admission_pediatric_covid_confirmed_SD',
       'previous_day_admission_pediatric_covid_suspected_SD',
       'staffed_adult_icu_bed_occupancy_SD',
       'staffed_adult_icu_bed_occupancy_coverage_SD',
       'staffed_icu_adult_patients_confirmed_and_suspected_covid_SD',
       'staffed_icu_adult_patients_confirmed_covid_SD',
       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_adult_patients_hospitalized_confirmed_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_covid_SD',
       'inpatient_beds_utilization_SD',
       'inpatient_beds_utilization_numerator_SD',
       'inpatient_beds_utilization_denominator_SD',
       'percent_of_inpatients_with_covid_SD',
       'inpatient_bed_covid_utilization_SD',
       'adult_icu_bed_covid_utilization_SD', 'adult_icu_bed_utilization_SD',
       'year','month','week','day']]
print(df.shape)

(375, 42)


Check Granger Causality of all possible combinations of the Time series.
The rows are the response variable, columns are predictors. 

The values in the table are the P-Values. 
P-Values lesser than the significance level (0.05), implies 
the Null Hypothesis is rejected (i.e the X does not cause Y can be rejected).

So, the alternate Hypothesis is correct and X causes Y.

In [77]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12 #timelags
test = 'ssr_chi2test'
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
    """
    data      : pandas dataframe containing the time series variables
    variables : list containing names of the time series variables.
    """
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

In [ ]:
grangers_causation_matrix(df, variables = df.columns)

,confirmed_cases_x,deaths_x,target_x,google_mobility_retail_and_recreation_x,google_mobility_grocery_and_pharmacy_x,google_mobility_workplaces_x,C_TOT_POP_x,google_mobility_residential_x,apple_mobility_driving_x,test_count_x,test_rate_x,new_test_rate_x,new_test_count_x,critical_staffing_shortage_today_yes_SD_x,critical_staffing_shortage_anticipated_within_week_yes_SD_x,hospital_onset_covid_SD_x,hospital_onset_covid_coverage_SD_x,inpatient_beds_SD_x,inpatient_beds_coverage_SD_x,previous_day_admission_adult_covid_confirmed_SD_x,previous_day_admission_adult_covid_suspected_SD_x,previous_day_admission_pediatric_covid_confirmed_SD_x,previous_day_admission_pediatric_covid_suspected_SD_x,staffed_adult_icu_bed_occupancy_SD_x,staffed_adult_icu_bed_occupancy_coverage_SD_x,staffed_icu_adult_patients_confirmed_and_suspected_covid_SD_x,staffed_icu_adult_patients_confirmed_covid_SD_x,total_adult_patients_hospitalized_confirmed_and_suspected_covid_SD_x,total_adult_patients_hospitalized_confirmed_covid_SD_x,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD_x,total_pediatric_patients_hospitalized_confirmed_covid_SD_x,inpatient_beds_utilization_SD_x,inpatient_beds_utilization_numerator_SD_x,inpatient_beds_utilization_denominator_SD_x,percent_of_inpatients_with_covid_SD_x,inpatient_bed_covid_utilization_SD_x,adult_icu_bed_covid_utilization_SD_x,adult_icu_bed_utilization_SD_x,year_x,month_x,week_x,day_x
confirmed_cases_y,1.0000,0.0412,0.0000,0.0000,0.0000,0.0007,1.0000,0.0000,0.1883,0.2660,0.2624,0.0252,0.0253,0.0675,0.0887,0.0003,0.0814,0.1384,0.1456,0.0000,0.3291,0.0001,0.0337,0.0031,0.0537,0.0044,0.0028,0.0000,0.0000,0.0179,0.0077,0.0443,0.0886,0.1763,0.0000,0.0000,0.0410,0.2375,0.0000,0.0000,0.0000,0.0336
deaths_y,0.0000,1.0000,0.0000,0.4604,0.5280,0.0047,0.0000,0.0175,0.0149,0.1736,0.1751,0.5126,0.5146,0.0000,0.0002,0.2526,0.1945,0.6473,0.5538,0.0566,0.0037,0.0799,0.0000,0.0008,0.0003,0.0727,0.0874,0.0576,0.1597,0.2095,0.3757,0.0329,0.0416,0.6055,0.2322,0.1840,0.0434,0.0023,0.0001,0.0011,0.0000,0.1908
target_y,0.0000,0.0000,1.0000,0.0000,0.0000,0.0005,0.0000,0.0000,0.1997,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0001,0.0004,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0145,0.0071,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0600
google_mobility_retail_and_recreation_y,0.0877,0.0094,0.0177,1.0000,0.0000,0.0000,0.0877,0.0000,0.0000,0.3477,0.3500,0.4242,0.4357,0.8187,0.5735,0.4416,0.2270,0.3705,0.1419,0.0004,0.8183,0.2230,0.5949,0.4151,0.1830,0.4834,0.4415,0.4494,0.4233,0.8759,0.9756,0.0000,0.0002,0.3585,0.0000,0.0328,0.5089,0.7667,0.2370,0.2870,0.5849,0.0245
google_mobility_grocery_and_pharmacy_y,0.0000,0.0000,0.0031,0.0000,1.0000,0.0068,0.0000,0.0000,0.0161,0.0000,0.0000,0.0120,0.0122,0.0000,0.0000,0.1235,0.0063,0.0135,0.0264,0.0000,0.0000,0.0164,0.0025,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0619,0.1178,0.0829,0.0001,0.0062,0.0009,0.0005,0.0000,0.0000,0.0011,0.3526,0.3453,0.0286
google_mobility_workplaces_y,0.0929,0.2106,0.0929,0.0000,0.0000,1.0000,0.0929,0.0069,0.0000,0.0780,0.0778,0.0052,0.0047,0.2725,0.3876,0.2895,0.0082,0.0033,0.0060,0.0003,0.4350,0.5739,0.3687,0.1656,0.0020,0.2635,0.2463,0.1738,0.2088,0.2122,0.2010,0.0000,0.0000,0.0077,0.0000,0.0018,0.0513,0.0204,0.0003,0.0001,0.0320,0.2432
C_TOT_POP_y,1.0000,0.0412,0.0000,0.0000,0.0000,0.0007,1.0000,0.0000,0.1883,0.2660,0.2624,0.0252,0.0253,0.0675,0.0887,0.0003,0.0814,0.1384,0.1456,0.0000,0.3291,0.0001,0.0337,0.0031,0.0537,0.0044,0.0028,0.0000,0.0000,0.0179,0.0077,0.0443,0.0886,0.1763,0.0000,0.0000,0.0410,0.2375,0.0000,0.0000,0.0000,0.0336
google_mobility_residential_y,0.0041,0.0125,0.0051,0.0000,0.0000,0.0012,0.0041,1.0000,0.0003,0.5119,0.5072,0.2271,0.2325,0.1498,0.1256,0.4337,0.0177,0.4506,0.0611,0.0000,0.2232,0.4389,0.0417,0.1066,0.0100,0.7282,0.8084,0.5650,0.4517,0.7531,0.9311,0.0386,0.0003,0.4644,0.0002,0.0850,0.6204,0.2693,0.0000,0.0000,0.0144,0.2870
apple_mobility_driving_y,0.2157,0.

We can see that the features like deaths, test_rate, test_count, previous day admission records,
total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD_y is very much causing the confirmed cases


Next, We will check the stationarity of our data.

We will use adfuller test, in which p values will determine the stationarity.

if p<0.05,  NULL Hypothesis is rejected and the data is stationary.

else NULL Hypothesis is selected and we can say that our data is Non-stationary

In [11]:
def adfuller_test(series, signif=0.05, name='', verbose=False):
    """Perform ADFuller to test for Stationarity of given series and print report"""
    r = adfuller(series, autolag='AIC')
    output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
    p_value = output['pvalue'] 
    def adjust(val, length= 6): return str(val).ljust(length)

    # Print Summary
    print(f'    Augmented Dickey-Fuller Test on "{name}"', "\n   ", '-'*47)
    print(f' Null Hypothesis: Data has unit root. Non-Stationary.')
    print(f' Significance Level    = {signif}')
    print(f' Test Statistic        = {output["test_statistic"]}')
    print(f' No. Lags Chosen       = {output["n_lags"]}')

    for key,val in r[4].items():
        print(f' Critical value {adjust(key)} = {round(val, 3)}')

    if p_value <= signif:
        print(f" => P-Value = {p_value}. Rejecting Null Hypothesis.")
        print(f" => Series is Stationary.")
    else:
        print(f" => P-Value = {p_value}. Weak evidence to reject the Null Hypothesis.")
        print(f" => Series is Non-Stationary.")  

In [14]:
# ADF Test on each column
for name, column in df.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "confirmed_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 3.3315
 No. Lags Chosen       = 10
 Critical value 1%     = -3.448
 Critical value 5%     = -2.87
 Critical value 10%    = -2.571
 => P-Value = 1.0. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "deaths" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = 2.9807
 No. Lags Chosen       = 14
 Critical value 1%     = -3.449
 Critical value 5%     = -2.87
 Critical value 10%    = -2.571
 => P-Value = 1.0. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "target" 
    -----------------------------------------------
 Null Hypothesis: Data has 

Since, there is a lots of non-stationarity in the data, we will apply differencing techinque to make it stationary

In [15]:
df_diff = df.diff().dropna()
for name, column in df_diff.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "confirmed_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -1.5656
 No. Lags Chosen       = 13
 Critical value 1%     = -3.449
 Critical value 5%     = -2.87
 Critical value 10%    = -2.571
 => P-Value = 0.5008. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "deaths" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -1.9572
 No. Lags Chosen       = 16
 Critical value 1%     = -3.449
 Critical value 5%     = -2.87
 Critical value 10%    = -2.571
 => P-Value = 0.3057. Weak evidence to reject the Null Hypothesis.
 => Series is Non-Stationary.


    Augmented Dickey-Fuller Test on "target" 
    -----------------------------------------------
 Null Hypothesis: D

After apply differencing one time, some variables are still non-stationary.

So again apply differencing and check.

In [16]:
df_diff = df_diff.diff().dropna()
for name, column in df_diff.iteritems():
    adfuller_test(column, name=column.name)
    print('\n')

    Augmented Dickey-Fuller Test on "confirmed_cases" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -8.6546
 No. Lags Chosen       = 12
 Critical value 1%     = -3.449
 Critical value 5%     = -2.87
 Critical value 10%    = -2.571
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "deaths" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance Level    = 0.05
 Test Statistic        = -7.8249
 No. Lags Chosen       = 15
 Critical value 1%     = -3.449
 Critical value 5%     = -2.87
 Critical value 10%    = -2.571
 => P-Value = 0.0. Rejecting Null Hypothesis.
 => Series is Stationary.


    Augmented Dickey-Fuller Test on "target" 
    -----------------------------------------------
 Null Hypothesis: Data has unit root. Non-Stationary.
 Significance L

Now, the data is stationary after applying differencing 2 times.

In [17]:
grangers_causation_matrix(df_diff, variables = df_diff.columns)

,confirmed_cases_x,deaths_x,target_x,google_mobility_retail_and_recreation_x,google_mobility_grocery_and_pharmacy_x,google_mobility_workplaces_x,C_TOT_POP_x,google_mobility_residential_x,apple_mobility_driving_x,test_count_x,test_rate_x,new_test_rate_x,new_test_count_x,critical_staffing_shortage_today_yes_SD_x,critical_staffing_shortage_anticipated_within_week_yes_SD_x,hospital_onset_covid_SD_x,hospital_onset_covid_coverage_SD_x,inpatient_beds_SD_x,inpatient_beds_coverage_SD_x,previous_day_admission_adult_covid_confirmed_SD_x,previous_day_admission_adult_covid_suspected_SD_x,previous_day_admission_pediatric_covid_confirmed_SD_x,previous_day_admission_pediatric_covid_suspected_SD_x,staffed_adult_icu_bed_occupancy_SD_x,staffed_adult_icu_bed_occupancy_coverage_SD_x,staffed_icu_adult_patients_confirmed_and_suspected_covid_SD_x,staffed_icu_adult_patients_confirmed_covid_SD_x,total_adult_patients_hospitalized_confirmed_and_suspected_covid_SD_x,total_adult_patients_hospitalized_confirmed_covid_SD_x,total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD_x,total_pediatric_patients_hospitalized_confirmed_covid_SD_x,inpatient_beds_utilization_SD_x,inpatient_beds_utilization_numerator_SD_x,inpatient_beds_utilization_denominator_SD_x,percent_of_inpatients_with_covid_SD_x,inpatient_bed_covid_utilization_SD_x,adult_icu_bed_covid_utilization_SD_x,adult_icu_bed_utilization_SD_x,year_x,month_x,week_x,day_x
confirmed_cases_y,1.0000,0.0000,0.0000,0.0000,0.0000,0.0002,1.0000,0.0000,0.2018,0.1764,0.1693,0.0440,0.0421,0.0757,0.0565,0.2929,0.0275,0.7833,0.1905,0.0000,0.6539,0.0001,0.0190,0.0049,0.2862,0.2615,0.2212,0.0558,0.0322,0.0095,0.0046,0.4717,0.3466,0.7165,0.6432,0.1077,0.2014,0.1290,0.0000,0.0000,0.0000,0.0375
deaths_y,0.0000,1.0000,0.0000,0.1652,0.2353,0.0007,0.0000,0.0016,0.0227,0.0356,0.0361,0.2276,0.2274,0.0081,0.0422,0.6025,0.0917,0.3170,0.3471,0.0142,0.7529,0.0675,0.0000,0.7073,0.7003,0.3920,0.3657,0.2630,0.1754,0.7167,0.3698,0.0067,0.0012,0.2803,0.3348,0.5433,0.3169,0.5439,0.0000,0.0001,0.0000,0.5310
target_y,0.0000,0.0000,1.0000,0.0000,0.0000,0.0003,0.0000,0.0000,0.2136,0.0002,0.0002,0.0009,0.0007,0.1170,0.0687,0.2748,0.0026,0.3724,0.0144,0.0000,0.6973,0.0002,0.0003,0.0025,0.0901,0.1937,0.2975,0.0408,0.0089,0.0014,0.0008,0.4827,0.3257,0.3769,0.4174,0.0220,0.2584,0.1500,0.0000,0.0000,0.0000,0.0393
google_mobility_retail_and_recreation_y,0.0027,0.0007,0.0030,1.0000,0.0000,0.0000,0.0027,0.0000,0.0000,0.0579,0.0583,0.2629,0.2650,0.3791,0.3481,0.2628,0.0948,0.3555,0.0200,0.0000,0.7625,0.1366,0.4995,0.1469,0.0531,0.7045,0.4336,0.4228,0.7037,0.9559,0.9472,0.0000,0.0000,0.2537,0.0000,0.0324,0.7186,0.6740,0.0606,0.0272,0.3384,0.0577
google_mobility_grocery_and_pharmacy_y,0.0063,0.0001,0.0063,0.0000,1.0000,0.0017,0.0063,0.0000,0.0010,0.0659,0.0660,0.1941,0.2003,0.3932,0.0892,0.3055,0.0760,0.1319,0.2472,0.0001,0.9135,0.6745,0.6222,0.2963,0.1547,0.4974,0.7073,0.2141,0.2622,0.5162,0.5108,0.0075,0.0001,0.1208,0.0003,0.2172,0.3714,0.2359,0.1647,0.1204,0.0860,0.0083
google_mobility_workplaces_y,0.0252,0.0241,0.0357,0.0000,0.0000,1.0000,0.0252,0.0120,0.0000,0.0421,0.0417,0.0019,0.0018,0.1222,0.3225,0.2374,0.0026,0.1048,0.0126,0.0001,0.8081,0.8141,0.1084,0.0332,0.0001,0.3114,0.2841,0.1650,0.2328,0.6634,0.6331,0.0001,0.0000,0.0417,0.0038,0.3161,0.0376,0.0095,0.0000,0.0000,0.0367,0.1312
C_TOT_POP_y,1.0000,0.0000,0.0000,0.0000,0.0000,0.0002,1.0000,0.0000,0.2018,0.1764,0.1693,0.0440,0.0421,0.0757,0.0565,0.2929,0.0275,0.7833,0.1905,0.0000,0.6539,0.0001,0.0190,0.0049,0.2862,0.2615,0.2212,0.0558,0.0322,0.0095,0.0046,0.4717,0.3466,0.7165,0.6432,0.1077,0.2014,0.1290,0.0000,0.0000,0.0000,0.0375
google_mobility_residential_y,0.0008,0.0008,0.0008,0.0000,0.0000,0.0000,0.0008,1.0000,0.0000,0.0692,0.0682,0.1221,0.1174,0.0515,0.0283,0.2909,0.0060,0.3488,0.0106,0.0000,0.9418,0.3874,0.0096,0.0529,0.0013,0.5706,0.6806,0.1400,0.2337,0.7125,0.8224,0.0061,0.0001,0.2968,0.0000,0.0739,0.4221,0.2583,0.0000,0.0000,0.0011,0.1714
apple_mobility_driving_y,0.6429,0.

In [1]:
# defined different types of metrics for validation data.

from sklearn.metrics import mean_squared_error,mean_absolute_error,median_absolute_error,r2_score

def validation_metrics(pred,true):
  report = {}
  pred = pred.diff().dropna()
  true = true.diff().dropna()
  pred = pred.astype('int64')
  report['mean_squared_error'] = mean_squared_error(pred,true)
  report['mean_absolute_error'] = mean_absolute_error(pred,true)
  report['root_mean_squared_error'] = math.sqrt(mean_squared_error(pred,true))
  report['median_absolute_error'] = median_absolute_error(pred,true)
  report['r2_score'] = r2_score(pred,true)
  #print('mean_squared_error',mean_squared_error(pred,true))
  #print('mean_absolute_error',mean_absolute_error(pred,true))
  #print('root_mean_squared_error',math.sqrt(mean_squared_error(pred,true)))
  #print('median_absolute_error',median_absolute_error(pred,true))
  #print('r2_score',r2_score(pred,true))
  return report

In [67]:
#checking for lags
df = data_by_county.get_group(1001)
df['date'] = pd.to_datetime(df['date'])
df = df.set_index(df['date'])

df = df[['confirmed_cases', 'deaths', 'google_mobility_retail_and_recreation',
       'google_mobility_grocery_and_pharmacy', 'google_mobility_workplaces',
       'google_mobility_residential',
       'new_test_rate', 'C_TOT_POP','new_test_count','critical_staffing_shortage_today_yes_SD',
       'critical_staffing_shortage_anticipated_within_week_yes_SD',
       'hospital_onset_covid_SD', 'hospital_onset_covid_coverage_SD',
       'previous_day_admission_adult_covid_confirmed_SD',
       'previous_day_admission_pediatric_covid_confirmed_SD',
       'previous_day_admission_pediatric_covid_suspected_SD',
       'staffed_adult_icu_bed_occupancy_SD',
       'staffed_adult_icu_bed_occupancy_coverage_SD',
       'staffed_icu_adult_patients_confirmed_and_suspected_covid_SD',
       'staffed_icu_adult_patients_confirmed_covid_SD',
       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_adult_patients_hospitalized_confirmed_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_covid_SD',
       'inpatient_beds_utilization_SD',
       'inpatient_beds_utilization_numerator_SD',
       'percent_of_inpatients_with_covid_SD',
       'inpatient_bed_covid_utilization_SD',
       'adult_icu_bed_covid_utilization_SD',
       'year','month','week','day']]
train = df['2020-01-22':'2021-01-10']
valid = df['2021-01-11':]

for lags in range(1,10):
      model = VAR(endog=train)
      model_fit = model.fit(lags)

      prediction = model_fit.forecast(model_fit.y, steps=20)
      cols = df.columns
      pred = pd.DataFrame(index=range(0,len(prediction)),columns=cols)
      for j in range(0,len(cols)):
          for i in range(0, len(prediction)):
            pred.iloc[i][j] = prediction[i][j]

      forecasted = pred['confirmed_cases'].reset_index().drop('index',axis=1)
      actual = valid['confirmed_cases']
      #print(forecasted)
      #print(actual)

      val = validation_metrics(forecasted,actual)

      print(lags,val['root_mean_squared_error'])

1 58.50775926949213
2 33.573956071869
3 36.86390332589663
4 69.56140793345266
5 33.781028621651565
6 133.81861250450768
7 1057.7311119860085
8 5303.750619286218
9 905.6967657990987


We can see that a lag of 2 gives us the best score, So we will use it in our training

In [68]:
def create_model(data_by_county,counties):
  cnt=0
  val_report = {}

  for c in counties:
    df = data_by_county.get_group(c) #grouping the data by county
    df['date'] = pd.to_datetime(df['date'])
    df = df.set_index(df['date'])

    df = df[['confirmed_cases', 'deaths', 'google_mobility_retail_and_recreation',
       'google_mobility_grocery_and_pharmacy', 'google_mobility_workplaces',
       'google_mobility_residential',
       'new_test_rate', 'C_TOT_POP','new_test_count','critical_staffing_shortage_today_yes_SD',
       'critical_staffing_shortage_anticipated_within_week_yes_SD',
       'hospital_onset_covid_SD', 'hospital_onset_covid_coverage_SD',
       'previous_day_admission_adult_covid_confirmed_SD',
       'previous_day_admission_pediatric_covid_confirmed_SD',
       'previous_day_admission_pediatric_covid_suspected_SD',
       'staffed_adult_icu_bed_occupancy_SD',
       'staffed_adult_icu_bed_occupancy_coverage_SD',
       'staffed_icu_adult_patients_confirmed_and_suspected_covid_SD',
       'staffed_icu_adult_patients_confirmed_covid_SD',
       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_adult_patients_hospitalized_confirmed_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_covid_SD',
       'inpatient_beds_utilization_SD',
       'inpatient_beds_utilization_numerator_SD',
       'percent_of_inpatients_with_covid_SD',
       'inpatient_bed_covid_utilization_SD',
       'adult_icu_bed_covid_utilization_SD',
       'year','month','week','day']] #select only those columns that will help in our prediction
    #split the data into training and validation
    train = df['2020-01-22':'2021-01-10']
    valid = df['2021-01-11':]

    model = VAR(endog=train) #defining the model
    model_fit = model.fit(2) #fit the model on training data

    prediction = model_fit.forecast(model_fit.y, steps=20) #making predictions on validation set
    cols = df.columns
    pred = pd.DataFrame(index=range(0,len(prediction)),columns=cols)
    for j in range(0,len(cols)):
        for i in range(0, len(prediction)):
          pred.iloc[i][j] = prediction[i][j]

    forecasted = pred['confirmed_cases'].reset_index().drop('index',axis=1)
    actual = valid['confirmed_cases']
    #print(forecasted)
    #print(actual)

    val_report[c] = validation_metrics(forecasted,actual) #calculating the metrics score on validation data
    cnt = cnt+1
    if cnt%100==0:
      print(cnt)
  return val_report

In [69]:
data_by_county = train_data.groupby('countyFIPS')
val_report1 = create_model(data_by_county,train_data['countyFIPS'].unique())

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100


In [71]:
#selecting some random 100 counties to see the validation score
import random
county = train_data['countyFIPS'].unique()
for i in range(100):
  c = random.choice(county)
  print(c,val_report1[c]['root_mean_squared_error'])

40051 24.98209885413743
21001 6.177207681213422
16057 13.024268845020377
26135 4.530597729822599
37195 47.48905691120628
48089 8.309791307785495
48319 5.947135533400226
53061 374.55924975756153
30023 4.454447806761001
29057 3.1455900626281967
51083 25.53841276030657
48349 68.17855813577818
8095 1.7917941611104424
37133 111.14736543033875
41009 10.587977293038305
31121 3.8044295512634103
48447 1.6059101370939324
37167 41.35278263404952
45059 39.01214385830459
51115 2.8469743865891624
17001 28.077899156678935
28081 23.645406808889856
13035 10.789858786452163
20081 1.9056702094980709
42051 66.10517934959488
5001 10.911124212424097
48503 21.458650568256296
27085 18.69843787452226
27013 12.90654668640768
30057 5.496410311811653
39019 7.6364775566804495
29229 10.533155672496854
48323 122.13883386066081
16063 2.752988806446741
18183 8.941329231219072
35015 30.510567625694957
17145 27.05355312786299
56041 12.137371604827967
40095 12.161198001928316
31011 2.115108557622118
20061 65.819929792538

In [72]:
#saving the validation report
import pickle
def save_obj(obj, name ):
    with open(''+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

save_obj(val_report1,'val_report1')

In [ ]:
#create a function that will take data and forecast for test data
def create_model(data_by_county,counties):
  X_test = pd.DataFrame(columns=['countyFIPS','stateFIPS','var_pred'])

  cnt=0
  for c in counties:
    df = data_by_county.get_group(c) #get data for county c
    state = df['stateFIPS'].iloc[-1]

    df = df[['confirmed_cases', 'deaths', 'google_mobility_retail_and_recreation',
       'google_mobility_grocery_and_pharmacy', 'google_mobility_workplaces',
       'google_mobility_residential',
       'new_test_rate', 'C_TOT_POP','new_test_count','critical_staffing_shortage_today_yes_SD',
       'critical_staffing_shortage_anticipated_within_week_yes_SD',
       'hospital_onset_covid_SD', 'hospital_onset_covid_coverage_SD',
       'previous_day_admission_adult_covid_confirmed_SD',
       'previous_day_admission_pediatric_covid_confirmed_SD',
       'previous_day_admission_pediatric_covid_suspected_SD',
       'staffed_adult_icu_bed_occupancy_SD',
       'staffed_adult_icu_bed_occupancy_coverage_SD',
       'staffed_icu_adult_patients_confirmed_and_suspected_covid_SD',
       'staffed_icu_adult_patients_confirmed_covid_SD',
       'total_adult_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_adult_patients_hospitalized_confirmed_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_and_suspected_covid_SD',
       'total_pediatric_patients_hospitalized_confirmed_covid_SD',
       'inpatient_beds_utilization_SD',
       'inpatient_beds_utilization_numerator_SD',
       'percent_of_inpatients_with_covid_SD',
       'inpatient_bed_covid_utilization_SD',
       'adult_icu_bed_covid_utilization_SD']]

    model = VAR(endog=df)     #defining the model
    model_fit = model.fit(2)  #fit the model on training data

    prediction = model_fit.forecast(model_fit.y, steps=15) #Making predictions on test data
    cols = df.columns
    pred = pd.DataFrame(index=range(0,len(prediction)),columns=cols)
    for j in range(0,len(cols)):
        for i in range(0, len(prediction)):
          pred.iloc[i][j] = prediction[i][j]

    #Saving the predictions in a dataframe  for submission in round 1.
    pre_test_var = pred['confirmed_cases'].reset_index().drop('index',axis=1)
    temp_df = pd.DataFrame({'countyFIPS':c,'stateFIPS':state,'var_pred':pre_test_var['confirmed_cases']})

    X_test = pd.concat([X_test,temp_df],axis=0)
    cnt = cnt+1
    if cnt%100==0:
      print(cnt)
  return X_test

In [ ]:
data_by_county = train_data.groupby('countyFIPS')
result = create_model(data_by_county,test_data['countyFIPS'].unique())

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100


So upto here, we trained our model on training data and made predictions on test data and saving the result in a result dataframe

In [ ]:
result.shape

(47130, 3)

In [ ]:
result = result.reset_index().drop('index',axis=1)

In [ ]:
sam_sub = pd.read_csv('EQ-2021-Submission-Template.csv')

In [ ]:
result.head()

,countyFIPS,stateFIPS,var_pred
0,1001,1,5605.98
1,1001,1,5627.74
2,1001,1,5654.93
3,1001,1,5685.43
4,1001,1,5717.68


In [ ]:
result['confirmed_cases'] = result['var_pred'].astype('int64')
result.drop(['var_pred'],axis=1)

,countyFIPS,stateFIPS,confirmed_cases
0,1001,1,5605
1,1001,1,5627
2,1001,1,5654
3,1001,1,5685
4,1001,1,5717
...,...,...,...
47125,56045,56,658
47126,56045,56,663
47127,56045,56,669
47128,56045,56,674


In [ ]:
sam_sub

,Unnamed: 0,countyFIPS,stateFIPS,date,Confirmed_cases
0,1,1001,1,2021-02-01,NaN
1,2,1001,1,2021-02-02,NaN
2,3,1001,1,2021-02-03,NaN
3,4,1001,1,2021-02-04,NaN
4,5,1001,1,2021-02-05,NaN
...,...,...,...,...,...
47125,47126,56045,56,2021-02-11,NaN
47126,47127,56045,56,2021-02-12,NaN
47127,47128,56045,56,2021-02-13,NaN
47128,47129,56045,56,2021-02-14,NaN


In [ ]:
unnamed = list(range(1,47131))

In [ ]:
#creating the required columns for submission
final_sub = pd.DataFrame({'Unnamed: 0':unnamed,'countyFIPS':result['countyFIPS'],'stateFIPS':result['stateFIPS'],'date':sam_sub['date'],'confirmed_cases':result['confirmed_cases']})

In [ ]:
final_sub.head(10)

,Unnamed: 0,countyFIPS,stateFIPS,date,confirmed_cases
0,1,1001,1,2021-02-01,5605
1,2,1001,1,2021-02-02,5627
2,3,1001,1,2021-02-03,5654
3,4,1001,1,2021-02-04,5685
4,5,1001,1,2021-02-05,5717
5,6,1001,1,2021-02-06,5750
6,7,1001,1,2021-02-07,5784
7,8,1001,1,2021-02-08,5819
8,9,1001,1,2021-02-09,5854
9,10,1001,1,2021-02-10,5891


In [ ]:
#Saving the predictions for our round 1 submission
final_sub.to_excel('Hack Elite_2018UGEC064_2018UGEC066_NIT Jamshedpur.xlsx')

In [ ]:
final_sub

,Unnamed: 0,countyFIPS,stateFIPS,date,confirmed_cases
0,1,1001,1,2021-02-01,5605
1,2,1001,1,2021-02-02,5627
2,3,1001,1,2021-02-03,5654
3,4,1001,1,2021-02-04,5685
4,5,1001,1,2021-02-05,5717
...,...,...,...,...,...
47125,47126,56045,56,2021-02-11,658
47126,47127,56045,56,2021-02-12,663
47127,47128,56045,56,2021-02-13,669
47128,47129,56045,56,2021-02-14,674
